In [127]:
#################################################################
#Developed by Jonathan Ojeda 20-01-2021 QAAFI-UQ
#Functionality:
#Convert from date to julian day
#Get soybean planting dates by US state using data from NASS-USDA
#Convert dtypes
#Locate specific rows and make a new dataframe
#create new rows with data
#Create campaig file for planting date
#Use of pivot table to trnspose data
#################################################################

In [1]:
import os
import sqlite3
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import dateutil
import pylab as py
import seaborn as sns
import scipy 
from scipy import stats
import sklearn.metrics
from numpy  import array
import glob
import functools
from functools import reduce
import matplotlib.ticker as ticker
from mpl_toolkits.axes_grid.inset_locator import (inset_axes, InsetPosition, mark_inset)
import statsmodels.api as sm
import matplotlib.patheffects as path_effects
import matplotlib.lines as mlines
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

<ipython-input-1-9e9d1d5ccf14>:19: MatplotlibDeprecationWarning: 
The mpl_toolkits.axes_grid module was deprecated in Matplotlib 2.1 and will be removed two minor releases later. Use mpl_toolkits.axes_grid1 and mpl_toolkits.axisartist, which provide the same functionality instead.
  from mpl_toolkits.axes_grid.inset_locator import (inset_axes, InsetPosition, mark_inset)


In [17]:
#read data
df=pd.read_csv(r'C:\Users\uqjojeda\Nextcloud\PURTERRA-A0131\2020\inputs\CornPdatesLatLonFinal.txt')
df2 = df.loc[(df['DOY'] > 0)]
df2.rename(columns={'Grids40K_2':'lat','Grids40K_3':'lon'}, inplace=True)
df2.drop(['FID', 'FID_latlon', 'FID_psims4', 'psims40kmp', 'pdates3_cs',
       'pdates3__1', 'pdates3__2', 'pdates3__3', 'pdates3__4', 'pdates3__5',
       'pdates3__6', 'pdates3__7', 'pdates3__8', 'pdates3__9', 'pdates3_10',
       'pdates3_11', 'pdates3_12', 'EmptyAreas', 'EmptyAre_1', 'EmptyAre_2',
       'FID_psimsT', 'psimsTiles', 'Grids40KmL', 'Grids40K_1',
       'Grids40K_4', 'FID_EmptyA', 'EmptyAre_3', 'EmptyAre_4', 'EmptyAre_5',
       'EmptyAre_6'], axis=1, inplace=True)

C:\Users\uqjojeda\Anaconda3\lib\site-packages\pandas\core\frame.py:4438: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
C:\Users\uqjojeda\Anaconda3\lib\site-packages\pandas\core\frame.py:4305: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [33]:
#read data
latlon = pd.read_csv(r'C:/Users/uqjojeda/Nextcloud/PURTERRA-A0131/2020/inputs/LatLonByState.csv')
tile = pd.read_csv(r'C:/Users/uqjojeda/Nextcloud/PURTERRA-A0131/2020/inputs/Grids40KmList.csv')
camp = pd.read_csv(r'C:/Users/uqjojeda/Nextcloud/PURTERRA-A0131/2020/inputs/CampaignLatLonList.csv')

In [34]:
#work only with jday50, lat and lon, columns
final = pd.merge(camp, df2, how="outer", on=["lat","lon"])
final

,lat,lon,DOY
0,-89.75,-179.75,NaN
1,-89.25,-179.75,NaN
2,-88.75,-179.75,NaN
3,-88.25,-179.75,NaN
4,-87.75,-179.75,NaN
...,...,...,...
259195,87.75,179.75,NaN
259196,88.25,179.75,NaN
259197,88.75,179.75,NaN
259198,89.25,179.75,NaN


In [36]:
final[['DOY']] = final[['DOY']].fillna(axis=1, value=127)
final[['DOY']] = final[['DOY']].round(decimals=0)

In [43]:
final2

lon,-179.75,-179.25,-178.75,-178.25,-177.75,-177.25,-176.75,-176.25,-175.75,-175.25,...,175.25,175.75,176.25,176.75,177.25,177.75,178.25,178.75,179.25,179.75
lat,,,,,,,,,,,,,,,,,,,,,
-89.75,127.0,127.0,127.0,127.0,127.0,127.0,127.0,127.0,127.0,127.0,...,127.0,127.0,127.0,127.0,127.0,127.0,127.0,127.0,127.0,127.0
-89.25,127.0,127.0,127.0,127.0,127.0,127.0,127.0,127.0,127.0,127.0,...,127.0,127.0,127.0,127.0,127.0,127.0,127.0,127.0,127.0,127.0
-88.75,127.0,127.0,127.0,127.0,127.0,127.0,127.0,127.0,127.0,127.0,...,127.0,127.0,127.0,127.0,127.0,127.0,127.0,127.0,127.0,127.0
-88.25,127.0,127.0,127.0,127.0,127.0,127.0,127.0,127.0,127.0,127.0,...,127.0,127.0,127.0,127.0,127.0,127.0,127.0,127.0,127.0,127.0
-87.75,127.0,127.0,127.0,127.0,127.0,127.0,127.0,127.0,127.0,127.0,...,127.0,127.0,127.0,127.0,127.0,127.0,127.0,127.0,127.0,127.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87.75,127.0,127.0,127.0,127.0,127.0,127.0,127.0,127.0,127.0,127.0,...,127.0,127.0,127.0,127.0,127.0,127.0,127.0,127.0,127.0,127.0
88.25,127.0,127.0,127.0,127.0,127.0,127.0,127.0,127.0,127.0,127.0,...,127.0,127.0,127.0,127.0,127.0,127.0,127.0,127.0,127.0,127.0
88.75,127.0,127.0,127.0,127.0,127.0,127.0,127.0,127.0,127.0,127.0,...,127.0,127.0,127.0,127.0,127.0,127.0,127.0,127.0,127.0,127.0


In [38]:
#Use pivot table to create the array for the campaign file in psims
final2 = final.pivot_table(values='DOY',index='lat',columns='lon', dropna=False)
final2
#final2.to_csv(r'C:\Users\uqjojeda\Nextcloud\PURTERRA-A0131\2020\inputs\pdateCampaign.csv')

### Remember to remove the lat and lon cells before to create the campaign file for pSIMS!!!!